In [6]:
from preparation import do_the_job
data = do_the_job(path="data/data.csv")
X = data['input']
Y = data['output']

X.describe()

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,nr.employed,job_blue-collar,job_entrepreneur,...,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success
count,41188.00000,41188.000000,41188.000000,41188.000000,4.118800e+04,4.118800e+04,4.118800e+04,4.118800e+04,41188.000000,41188.000000,...,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,2.567593,962.475454,0.172963,7.797546e-17,-3.315820e-14,-1.352495e-16,-1.024860e-14,0.224677,0.035350,...,0.334296,0.099568,0.017432,0.013839,0.206711,0.209357,0.196416,0.197485,0.863431,0.033335
std,10.42125,2.770014,186.910907,0.494901,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,0.417375,0.184665,...,0.471750,0.299427,0.130877,0.116824,0.404951,0.406855,0.397292,0.398106,0.343396,0.179512
min,17.00000,1.000000,0.000000,0.000000,-2.216433e+00,-2.374889e+00,-2.224953e+00,-2.815697e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32.00000,1.000000,999.000000,0.000000,-1.197935e+00,-8.649547e-01,-4.747909e-01,-9.402810e-01,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,38.00000,2.000000,999.000000,0.000000,6.480923e-01,2.994571e-01,-2.803284e-01,3.316799e-01,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,47.00000,3.000000,999.000000,0.000000,8.390606e-01,7.227225e-01,8.864466e-01,8.451701e-01,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,98.00000,56.000000,999.000000,7.000000,8.390606e-01,2.058168e+00,2.939106e+00,8.451701e-01,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
positive = round(Y.sum()/len(Y) * 100)
negative = 100 - positive
print(f'True: {positive}%, Negative:{negative}%')

True: 11%, Negative:89%


In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn import metrics
from utils import cv_zoom

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42, stratify=Y)
num_folds = 5
cv = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Hyperparamètres et nombre de folds pour la validation croisée
param_grid = {
    "max_depth": [2, 6, 10],  # Largeur des arbres
    "n_estimators": [25, 50, 150, 250],  # Nombre d'arbres
    "learning_rate": [0.3, 0.5],  # Taux d'apprentissage
    "subsample": [0.3, 0.5],  # Ratio de sous-échantillonnage
    'min_samples_split': [2, 8, 15],
}

gb = GradientBoostingClassifier(random_state=42)

scoring = 'precision'
grid_search = GridSearchCV(
    estimator=gb,
    param_grid=param_grid,
    cv=cv,
    scoring=scoring,
)
grid_search.fit(X_train, y_train)

# Les meilleurs hyperparamètres
best_params = grid_search.best_params_
print("Meilleurs hyperparamètres : ", best_params)
best_gb = GradientBoostingClassifier(random_state=42, **best_params)

Training checkout

In [ ]:
cv_zoom(model=best_gb, X_train=X_train, y_train=y_train, num_folds=num_folds)

NameError: name 'best_gb' is not defined

Testing checkout

In [ ]:
best_gb_model = grid_search.best_estimator_
best_gb_model.fit(X_train, y_train)
y_pred = best_gb_model.predict(X_test)

precision = metrics.precision_score(y_test, y_pred)
print(f"Précision : {precision}")

tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_pred).ravel()
results = {
    'True': [tp, fn],
    'False': [fp, tn]
}
results = pd.DataFrame(results, index=['Predicted True', 'Predicted False'])
results

# Extreme Gradient Boosting Classifier

In [ ]:
import xgboost as xgb

param_grid = {
    "max_depth": [4, 6, 10],  # Largeur des arbres
    "n_estimators": [25, 50, 100],  # Nombre d'arbres
    "learning_rate": [0.1, 0.5],  # Taux d'apprentissage
    "subsample": [0.3, 0.5],  # Ratio de sous-échantillonnage
}

xgb_model = xgb.XGBClassifier(
    tree_method='exact',
    eval_metric="error", # map (mean average precision), logloss ou error
    #scale_pos_weight=np.sqrt(negative/positive) # donne plus de poids aux positives
)

clf = GridSearchCV(
    xgb_model,
    param_grid=param_grid,
    cv=cv,
)
clf.fit(X_train, y_train)
print(clf.best_score_)
print(clf.best_params_)

0.899552629421612
{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 25, 'subsample': 0.5}


Training checkout

In [ ]:
cv_zoom(model=clf.best_estimator_, X_train=X_train, y_train=y_train, num_folds=num_folds)
best_xgb_model = clf.best_estimator_
best_xgb_model.fit(X_train, y_train)
print(best_xgb_model.predict(X_train))

NameError: name 'clf' is not defined

Testing checkout

In [ ]:
y_pred = best_xgb_model.predict(X_test)

precision = metrics.precision_score(y_test, y_pred)
print(f"Précision : {precision}")

tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_pred).ravel()
results = {
    'True': [tp, fn],
    'False': [fp, tn]
}
results = pd.DataFrame(results, index=['Predicted True', 'Predicted False'])
results

Précision : 0.706766917293233


,True,False
Predicted True,282,117
Predicted False,1110,10848


# Test bizarre

In [ ]:
y_pred_bis = best_xgb_model.predict(X_train)

In [ ]:
condition = (y_pred_bis != 1) & (y_train != 1) # on enlève tous les vrais négatifs
X_train_bis = X_train.loc[condition]
y_train_bis = y_train[condition]
len(X_train_bis)

25323

In [ ]:
param_grid = {
    "max_depth": [2, 4, 6, 7, 8, 9, 10],  # Largeur des arbres
    "n_estimators": [25, 50, 100],  # Nombre d'arbres
    "learning_rate": [0.1, 0.2, 0.5, 0.7],  # Taux d'apprentissage
    "subsample": [0.3, 0.5, 0.7, 0.9],  # Ratio de sous-échantillonnage
}

xgb_model = xgb.XGBClassifier(
    tree_method='exact',
    eval_metric="error", # map (mean average precision), logloss ou error
    #scale_pos_weight=np.sqrt(negative/positive) # donne plus de poids aux positives
)

clf = GridSearchCV(
    xgb_model,
    param_grid=param_grid,
    cv=cv,
)
clf.fit(X_train_bis, y_train_bis)
print(clf.best_score_)
print(clf.best_params_)
cv_zoom(model=clf.best_estimator_, X_train=X_train, y_train=y_train, num_folds=num_folds)

best_xgb_model = clf.best_estimator_
best_xgb_model.fit(X_train, y_train)
y_pred = best_xgb_model.predict(X_test)

precision = metrics.precision_score(y_test, y_pred)
print(f"Précision : {precision}")

tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_pred).ravel()
results = {
    'True': [tp, fn],
    'False': [fp, tn]
}
results = pd.DataFrame(results, index=['Predicted True', 'Predicted False'])
results

1.0
{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 25, 'subsample': 0.3}
Fold  : precision  | accuracy   | recall    
Fold 1:      0.719 |      0.901 |      0.197
Fold 2:      0.669 |      0.898 |       0.18
Fold 3:      0.648 |      0.897 |      0.179
Fold 4:      0.677 |      0.897 |      0.168
Fold 5:      0.709 |        0.9 |      0.195
Précision : 0.7477744807121661


,True,False
Predicted True,252,85
Predicted False,1140,10880
